# CISC 452 Assignment 3 - Unsupervised Learning (100 points)  

Please put your name and student id

    Yanzhang Ma, #20090412

- The notebook file has clearly marked blocks where you are expected to write code. Do not write or modify any code outside of these blocks.
- Do not add or delete cells from the notebook.
- Run all cells, and do not clear out the outputs, before submitting. You will only get credit for code that has been run.
- Make sure to run all the cells from beginning before the submission
- Mark will be deducted based on late policy (-10% per day after due date until the end date after which no assignments will be accepted)

## [Part 1 Simple Competitive Learning (50 points)](#Part-1-Simple-Competitive-Learning)  

Build a **Kohonen Network** and use **Maxnet** at the output layer to find the node with the highest activation.  

### Build Model (30 points)  
Kohonen Network (20 points)  
Maxnet (10 points)  

### Evaluate Model (20 points)  
Use the Kohonen Network to predict the lables with the train and test datasets (10 points)  
Evaluate the prediction results (10 points)  
- Evaluation matrics include confusion matrix and accuracy

## [Part 2 Principle Component Analysis (50 points)](#Part-2-Principle-Component-Analysis)

Implement a **PCA Network** (not PCA) to reduce the dimension of the Iris dataset from 4 to 3.  
Use the Kohonen Network in Part 1 to train and test on the new dataset.  

### Build Model (30 points)  
Build PCA Network (20 points)  
Train the PCA model and obtain the new datasets with reduced dimension (10 points)  

### Evaluate Model (20 points)  
Use the Kohonen Network to predict the lables with the new train and test datasets (10 points)  
Evaluate the prediction results (10 points)  
- Evaluation matrics include confusion matrix and accuracy

In [51]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [52]:
data = load_iris()
x = data.data
x = (x - x.mean(axis=0)) / x.std(axis=0)
y = data.target
data.feature_names, data.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [54]:
def evaluator(y_test, y_pred):
    ####################################################################################################
    # enter code here to implement the evaluation matrices including confusion matrix and accuracy
    # You can only use Numpy here
    cm00=0
    cm01=0
    cm02=0
    cm10=0
    cm11=0
    cm12=0
    cm20=0
    cm21=0
    cm22=0
    for i,j in zip(y_test, y_pred):
        if i==j==0:
            cm00+=1
        elif (i==0) and (j==1):
            cm01+=1
        elif (i==0) and (j==2):
            cm02+=1
        elif (i==1) and (j==0):
            cm10+=1
        elif (i==1) and (j==1):
            cm11+=1
        elif (i==1) and (j==2):
            cm12+=1
        elif (i==2) and (j==0):
            cm20+=1
        elif (i==2) and (j==1):
            cm21+=1
        elif (i==2) and (j==2):
            cm22+=1
            
    confusion_matrix = [[cm00,cm01,cm02],[cm10,cm11,cm12],[cm20,cm21,cm22]]
    print("Confusion Matrix:\n",confusion_matrix)
    accuracy = np.mean(np.equal(np.array(y_pred),np.array(y_test)))
    print("accuracy:\n",accuracy)

    ####################################################################################################

In [55]:
# build a baseline model with K-Means
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3) #n_clusters - the number of clusters
km.fit(x_train)
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)

Confusion Matrix:
 [[40, 0, 0], [0, 8, 32], [0, 26, 14]]
accuracy:
 0.5166666666666667
Confusion Matrix:
 [[10, 0, 0], [0, 3, 7], [0, 7, 3]]
accuracy:
 0.5333333333333333


## Part 1 Simple Competitive Learning

In [56]:
####################################################################################################
# enter code here to build the Kohonen Network with Maxnet
class Kohonen(object):
    def __init__(self):
      self.output = 3
      self.learning_rate = 0.2
      self.bias = 1

    
    def train(self, x, n_epoch=20):
      n_train, input = x.shape
      self.weights = np.random.random((self.output, input))
      for i in range(n_epoch):
        for xi in x:
          d_1 = np.linalg.norm(xi-self.weights[0])
          d_2 = np.linalg.norm(xi-self.weights[1])
          d_3 = np.linalg.norm(xi-self.weights[2])
          vector=np.array([d_1,d_2,d_3])
          winner = self.Maxnet(vector)
          for w in range(len(self.weights[winner])):
            d = xi[w] - self.weights[winner][w]
            self.weights[winner][w]+=self.learning_rate*d
    
    def predict(self, x):
      output = []
      for i in x:
        d_1 = np.linalg.norm(i-self.weights[0])
        d_2 = np.linalg.norm(i-self.weights[1])
        d_3 = np.linalg.norm(i-self.weights[2]) 
        vector = np.array([d_1,d_2,d_3])
        winner = self.Maxnet(vector)
        output.append(winner)
      y_pred = output
      return y_pred
    
    def Maxnet(self, vector):
      while True:
        for i in range(np.size(vector)):
          vector[i] = max(0, vector[i] - 1/np.size(vector)*np.sum(np.delete(vector,i)))
        if np.size(np.nonzero(vector)) == 1:
          break
      return np.nonzero(vector)[0][0]



####################################################################################################

In [57]:
####################################################################################################
# enter code here to train and test the Kohonen Network
kohonen = Kohonen()
kohonen.train(x_train, n_epoch = 20)
y_pred = kohonen.predict(x_train)
evaluator(y_train, y_pred)
y_pred = kohonen.predict(x_test)
evaluator(y_test, y_pred)

####################################################################################################

Confusion Matrix:
 [[0, 0, 40], [0, 6, 34], [0, 0, 40]]
accuracy:
 0.38333333333333336
Confusion Matrix:
 [[0, 0, 10], [0, 0, 10], [0, 0, 10]]
accuracy:
 0.3333333333333333


## Part 2 Principle Component Analysis

In [106]:
####################################################################################################
# enter code here to build the PCA model
class PCA(object):
    def __init__(self, learning_rate=0.2, input_feature=4, output_feature=3):
      self.learning_rate = learning_rate
      self.weights= np.random.random((output_feature, input_feature))
    
    def train(self, X, n_epoch = 10):
      for i in range(n_epoch):
        for x in X:
          output = np.matmul(self.weights, x)
          for a in range(len(output)):
            for b in range(len(x)):
              d=output[a]*(x[b] - np.sum(output*np.transpose(self.weights)[b]))
              feature=self.learning_rate*d
              self.weights[a][b]+=feature
    
    def predict(self, X):
      output = []
      for i in X:
        X_feature = np.matmul(self.weights, i)
        output.append(X_feature)
      return np.array(output)


####################################################################################################

In [107]:
####################################################################################################
# enter code here to train the PCA model and obtain the new train and test datasets
pca=PCA()
pca.train(x_train)
x_train_PCA=pca.predict(x_train)
x_test_PCA=pca.predict(x_test)
print('Final weight is:\n', pca.weights)

####################################################################################################

Final weight is:
 [[-0.46183731  0.8027779   0.1042786   0.36870339]
 [ 0.45099482 -0.08745199  0.68591993  0.5720076 ]
 [ 0.73350428  0.583829   -0.14649012 -0.31399076]]


In [108]:
####################################################################################################
# enter code here to train and test the Kohonen Network with the new datasets
trained_kohonen = Kohonen()
trained_kohonen.train(x_train_PCA, n_epoch = 10)
y_pred = trained_kohonen.predict(x_train_PCA)
evaluator(y_train, y_pred)
y_pred = trained_kohonen.predict(x_test_PCA)
evaluator(y_test, y_pred)

####################################################################################################

Confusion Matrix:
 [[0, 0, 40], [0, 6, 34], [0, 0, 40]]
accuracy:
 0.38333333333333336
Confusion Matrix:
 [[0, 0, 10], [0, 1, 9], [0, 0, 10]]
accuracy:
 0.36666666666666664
